#  Silver to Gold Layer Notebook

##  Objective:
This notebook creates meaningful analytical tables from the cleaned Silver layer data and exports them to the Gold container and Azure SQL for reporting.

---

##  Step 1: Start Spark Session
Initialize Spark to execute SQL and PySpark operations.

---

##  Step 2: Read Silver Data
Read Silver-layer Parquet files into DataFrames for processing.

---

##  Step 3: Create Temp Views
Convert DataFrames into temporary views so SQL queries can be used for aggregations.

---

##  Step 4: Run Analytical Queries

###  Team Performance Metrics
- Total Wins
- Total Matches
- Win %
- Home vs Away Win %
- Tie/Cancelled %

###  Player Contribution
- Runs Scored
- Wickets Taken
- Top 2 contributors per team

###  Player Efficiency
- Average Strike Rate
- Batting Runs
- Bowling Wickets

###  Orange Cap & Purple Cap
- Highest run scorer and wicket taker (with KPIs)

###  Venue Analysis
- Home Stadium Performance
- Away Performance

---

##  Step 5: Save Aggregated Tables
Each SQL result is saved as a DataFrame and then written:
- To the **Gold container** in Parquet format.
- To the **Azure SQL Database** using JDBC.

This enables:
- Historical storage in ADLS.
- Real-time querying from Power BI via SQL.


In [0]:
# JDBC connection string
jdbc_url = f"jdbc:sqlserver://ipldata123.database.windows.net:1433;databaseName=ipl_data"
 
# Connection properties
connection_properties = {
    "user": "satyamsingh",
    "password": "Satyam@961",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
 
 

In [0]:
df_match_performance = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "silver_db.match_performance") \
    .options(**connection_properties) \
    .load()
 
df_match_performance.createOrReplaceTempView("match_performance")
 

In [0]:
df_match_stadium = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "silver_db.match_stadium") \
    .options(**connection_properties) \
    .load()

df_match_stadium.createOrReplaceTempView("match_stadium")

In [0]:
df_player_performance = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "silver_db.player_performance") \
    .options(**connection_properties) \
    .load()

df_player_performance.createOrReplaceTempView("player_performance")

In [0]:
df_player_team = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "silver_db.player_team") \
    .options(**connection_properties) \
    .load()

df_player_team.createOrReplaceTempView("player_team")

In [0]:
df_stadium = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "silver_db.stadium") \
    .options(**connection_properties) \
    .load()

df_stadium.createOrReplaceTempView("stadium")

In [0]:
df_team = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "silver_db.team") \
    .options(**connection_properties) \
    .load()

df_team.createOrReplaceTempView("team")

In [0]:
%sql
WITH match_results AS (
    SELECT team_id AS team, match_result FROM match_performance
    UNION ALL
    SELECT opponent_team_id AS team,
        CASE 
            WHEN match_result = 'Win' THEN 'Loss'
            WHEN match_result = 'Loss' THEN 'Win'
            ELSE match_result
        END AS match_result
    FROM match_performance
)
SELECT 
    t.team_name,
    COUNT(CASE WHEN mr.match_result = 'Win' THEN 1 END) AS Wins,
    COUNT(CASE WHEN mr.match_result = 'Loss' THEN 1 END) AS Losses,
    COUNT(CASE WHEN mr.match_result = 'Tie' THEN 1 END) AS Ties
FROM match_results mr
JOIN team t ON t.team_id = mr.team
GROUP BY t.team_name
ORDER BY t.team_name;

team_name,Wins,Losses,Ties
Chennai Super Kings,3,1,1
Delhi Capitals,3,6,1
Kolkata Knight Riders,3,2,3
Mumbai Indians,5,3,3
Sunrisers Hyderabad,1,3,0


In [0]:
Team_performance_metrics = _sqldf

In [0]:
Team_performance_metrics.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "gold_db.Team_performance_metrics") \
    .options(**connection_properties) \
    .mode("OVERWRITE") \
    .save()
 

In [0]:
%sql
WITH all_matches AS (
    SELECT 
        match_performance_match_id AS match_id,
        team_id AS team,
        opponent_team_id AS opponent,
        match_result
    FROM match_performance

    UNION ALL

    SELECT 
        match_performance_match_id AS match_id,
        opponent_team_id AS team,
        team_id AS opponent,
        CASE 
            WHEN match_result = 'Win' THEN 'Loss'
            WHEN match_result = 'Loss' THEN 'Win'
            ELSE match_result
        END AS match_result
    FROM match_performance
),
match_with_venue AS (
    SELECT 
        am.match_id,
        am.team,
        am.match_result,
        t.home_ground,
        s.stadium_name AS match_venue
    FROM all_matches am
    JOIN team t ON am.team = t.team_id
    JOIN match_stadium ms ON am.match_id = ms.match_id
    JOIN stadium s ON ms.stadium_id = s.stadium_id
),
final_stats AS (
    SELECT 
        t.team_name,
        CASE 
            WHEN mwv.home_ground = mwv.match_venue THEN 'Home'
            ELSE 'Away'
        END AS venue_type,
        mwv.match_result
    FROM match_with_venue mwv
    JOIN team t ON mwv.team = t.team_id
)
SELECT 
    team_name,
    venue_type,
    COUNT(CASE WHEN match_result = 'Win' THEN 1 END) AS Wins,
    COUNT(CASE WHEN match_result = 'Loss' THEN 1 END) AS Losses,
    COUNT(CASE WHEN match_result = 'Tie' THEN 1 END) AS Ties,
    COUNT(*) AS Total_Matches
FROM final_stats
GROUP BY team_name, venue_type
ORDER BY team_name, venue_type;

team_name,venue_type,Wins,Losses,Ties,Total_Matches
Chennai Super Kings,Away,1,1,1,3
Chennai Super Kings,Home,2,0,0,2
Delhi Capitals,Away,2,6,1,10
Delhi Capitals,Home,1,0,0,1
Kolkata Knight Riders,Away,3,1,2,6
Kolkata Knight Riders,Home,0,1,1,2
Mumbai Indians,Away,4,3,1,8
Mumbai Indians,Home,1,0,2,3
Sunrisers Hyderabad,Away,0,3,0,4
Sunrisers Hyderabad,Home,1,0,0,1


In [0]:
venue_analysis = _sqldf

In [0]:
venue_analysis.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "gold_db.venue_analysis") \
    .options(**connection_properties) \
    .mode("OVERWRITE") \
    .save()

In [0]:
%sql
WITH player_stats AS (
    SELECT 
        pp.player_id,
        pt.player_name,
        pt.player_role,
        pt.player_team_id,
        t.team_name,
        pp.player_runs_scored,
        pp.player_wickets_taken,
        pp.ball_taken,
        CASE 
            WHEN pp.ball_taken <> 0 
                THEN CAST(pp.player_runs_scored * 100.0 AS FLOAT) / pp.ball_taken
            ELSE NULL
        END AS strike_rate
    FROM player_performance pp
    JOIN player_team pt 
        ON pp.player_id = pt.player_id
    JOIN team t
        ON pt.player_team_id = t.team_id
)

SELECT
    player_id,
    player_name,
    player_role,
    team_name,
    ROUND(AVG(strike_rate), 2) AS avg_strike_rate,
    SUM(player_runs_scored) AS total_runs,
    SUM(player_wickets_taken) AS total_wickets
FROM player_stats
GROUP BY player_id, player_name, player_role, team_name
ORDER BY total_runs DESC;

player_id,player_name,player_role,team_name,avg_strike_rate,total_runs,total_wickets
102,Laurie Brooks,Batsman,Sunrisers Hyderabad,194.0,97,2
108,Jared Knox,Bowler,Chennai Super Kings,188.0,94,4
109,Kevin Conrad,Bowler,Chennai Super Kings,186.0,93,2
115,Breanna Jackson,All-rounder,Kolkata Knight Riders,170.0,85,0
101,Thomas Mendez,All-rounder,Mumbai Indians,164.0,82,2
114,Joshua Baird,Bowler,Chennai Super Kings,162.0,81,1
118,Michael Martinez,All-rounder,Delhi Capitals,160.0,80,1
120,Justin White,Batsman,Mumbai Indians,150.0,75,0
110,Alan Padilla,All-rounder,Kolkata Knight Riders,150.0,75,3
113,Joseph Meadows,Batsman,Sunrisers Hyderabad,140.0,70,1


In [0]:
Player_Efficiency_Metrics=_sqldf 

In [0]:
Player_Efficiency_Metrics.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "gold_db.Player_Efficiency_Metrics") \
    .options(**connection_properties) \
    .mode("OVERWRITE") \
    .save()

In [0]:
%sql
WITH all_match_results AS (
    SELECT 
        team_id AS team, 
        match_performance_match_id AS match_id, 
        match_result 
    FROM match_performance

    UNION ALL

    SELECT 
        opponent_team_id AS team, 
        match_performance_match_id,
        CASE 
            WHEN match_result = 'Win' THEN 'Loss'
            WHEN match_result = 'Loss' THEN 'Win'
            ELSE match_result
        END AS match_result
    FROM match_performance
),

match_with_stadium AS (
    SELECT 
        amr.team,
        amr.match_id,
        amr.match_result,
        t.team_name,
        t.home_ground,
        s.stadium_name
    FROM all_match_results amr
    JOIN team t ON amr.team = t.team_id
    JOIN match_stadium ms ON amr.match_id = ms.match_id
    JOIN stadium s ON ms.stadium_id = s.stadium_id
),

location_tagged AS (
    SELECT *,
        CASE 
            WHEN stadium_name = home_ground THEN 'Home'
            ELSE 'Away'
        END AS match_location
    FROM match_with_stadium
),

win_stats AS (
    SELECT 
        team_name,
        COUNT(*) AS total_matches,

        -- Overall counts
        COUNT(CASE WHEN match_result = 'Win' THEN 1 END) AS total_wins,
        COUNT(CASE WHEN match_result = 'Loss' THEN 1 END) AS total_losses,
        COUNT(CASE WHEN match_result = 'Tie' THEN 1 END) AS total_ties,
        COUNT(CASE WHEN match_result = 'Cancelled' THEN 1 END) AS total_cancelled,

        
        COUNT(CASE WHEN match_location = 'Home' THEN 1 END) AS home_matches,
        COUNT(CASE WHEN match_location = 'Away' THEN 1 END) AS away_matches,
        COUNT(CASE WHEN match_result = 'Win' AND match_location = 'Home' THEN 1 END) AS home_wins,
        COUNT(CASE WHEN match_result = 'Win' AND match_location = 'Away' THEN 1 END) AS away_wins
    FROM location_tagged
    GROUP BY team_name
)

SELECT 
    team_name,
    total_matches,

    
    ROUND(CAST(total_wins AS FLOAT) / total_matches * 100, 2) AS win_percentage,
    ROUND(CAST(total_losses AS FLOAT) / total_matches * 100, 2) AS loss_percentage,
    ROUND(CAST(total_ties AS FLOAT) / total_matches * 100, 2) AS tie_percentage,
    ROUND(CAST(total_cancelled AS FLOAT) / total_matches * 100, 2) AS cancelled_percentage,

    -- Home and Away win %
    ROUND(CAST(home_wins AS FLOAT) / NULLIF(home_matches, 0) * 100, 2) AS home_win_percentage,
    ROUND(CAST(away_wins AS FLOAT) / NULLIF(away_matches, 0) * 100, 2) AS away_win_percentage

FROM win_stats
ORDER BY win_percentage DESC;

team_name,total_matches,win_percentage,loss_percentage,tie_percentage,cancelled_percentage,home_win_percentage,away_win_percentage
Chennai Super Kings,5,60.0,20.0,20.0,0.0,100.0,33.33
Mumbai Indians,11,45.45,27.27,27.27,0.0,33.33,50.0
Kolkata Knight Riders,8,37.5,25.0,37.5,0.0,0.0,50.0
Delhi Capitals,11,27.27,54.55,9.09,9.09,100.0,20.0
Sunrisers Hyderabad,5,20.0,60.0,0.0,20.0,100.0,0.0


In [0]:
Additional_Metrics=_sqldf

In [0]:
Additional_Metrics.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "gold_db.Additional_Metrics") \
    .options(**connection_properties) \
    .mode("OVERWRITE") \
    .save()

In [0]:
%sql
WITH player_stats AS (
    SELECT 
        tm.team_name,
        pt.player_name,
        pt.player_role,
        SUM(pp.player_runs_scored) AS total_runs,
        SUM(pp.player_wickets_taken) AS total_wickets,
        SUM(pp.ball_taken) AS total_balls,
        ROUND(
            CASE 
                WHEN SUM(pp.ball_taken) > 0 THEN 
                    (CAST(SUM(pp.player_runs_scored) AS FLOAT) / SUM(pp.ball_taken)) * 100 
                ELSE NULL 
            END, 2
        ) AS strike_rate,
        -- Contribution Score: runs + (10 * wickets)
        SUM(pp.player_runs_scored) + 10 * SUM(pp.player_wickets_taken) AS contribution_score
    FROM player_performance pp
    JOIN player_team pt ON pp.player_id = pt.player_id
    JOIN team tm ON pt.player_team_id = tm.team_id
    GROUP BY tm.team_name, pt.player_name, pt.player_role
),
ranked_stats AS (
    SELECT *,
        ROW_NUMBER() OVER (PARTITION BY team_name ORDER BY contribution_score DESC) AS rn
    FROM player_stats
)
SELECT 
    team_name,
    player_name,
    player_role,
    total_runs,
    total_wickets,
    strike_rate,
    contribution_score
FROM ranked_stats
WHERE rn <= 2
ORDER BY team_name, rn;


team_name,player_name,player_role,total_runs,total_wickets,strike_rate,contribution_score
Chennai Super Kings,Jared Knox,Bowler,94,4,188.0,134
Chennai Super Kings,Kevin Conrad,Bowler,93,2,186.0,113
Delhi Capitals,Michael Martinez,All-rounder,80,1,160.0,90
Delhi Capitals,Autumn Chang,All-rounder,68,2,136.0,88
Kolkata Knight Riders,Alan Padilla,All-rounder,75,3,150.0,105
Kolkata Knight Riders,Breanna Jackson,All-rounder,85,0,170.0,85
Mumbai Indians,Thomas Mendez,All-rounder,82,2,164.0,102
Mumbai Indians,Bruce Morales,All-rounder,58,4,116.0,98
Sunrisers Hyderabad,Laurie Brooks,Batsman,97,2,194.0,117
Sunrisers Hyderabad,Joseph Meadows,Batsman,70,1,140.0,80


In [0]:
Player_Contribution=_sqldf

In [0]:
Player_Contribution.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "gold_db.Player_Contribution") \
    .options(**connection_properties) \
    .mode("OVERWRITE") \
    .save()